Based on these posts: 
* https://medium.com/@chrisfotache/text-classification-in-python-pipelines-nlp-nltk-tf-idf-xgboost-and-more-b83451a327e0
* https://github.com/keisukeirie/Amazon_review_helpfulness_prediction
* https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/
* https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22

In [ ]:
!pip install -q boto3

In [5]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [6]:
!head ./data/train_transformed.csv

0.0,0.1590044463957165,0.19972650901791134,-0.05066662776312519,0.16409915060964128,-0.030212000062510017,-0.010093728038304566,0.02327698986208724,-0.028741750360576773,-0.030597132792245716,0.07027875768729384,0.006704369354750269,0.03646038874737715,-0.0025510106486922313,0.03718889467180332,0.029598881966913185,-0.03392642549243057,-0.06332830039246741,-0.04471643226735182,0.12634402047495308,0.02874023403542355,0.09110297965834793,0.08891025146194247,0.07522121256771525,0.07106941171488831,0.12079685517937944,0.2061116468067043,0.12594330215863153,-0.02666756620664528,0.18138184474899555,0.06156695921744706,-0.135223326843388,-0.08460052425220384,-0.05269636065992008,-0.077649354395182,-0.01021643430248207,0.0226875726826212,-0.002539214302769293,4.5175908694184965e-06,-5.415988382898133e-05,-0.005303960687956904,-0.039204303898424324,-0.04865034420657565,0.02580859836147309,-0.07234657826598392,-0.03414986611395313,-0.02187350549993397,0.015512376105909012,-0.009209236518833893,0

In [7]:
!head ./data/validation_transformed.csv

0.0,0.037892980764537815,0.013901784329633984,0.1194632962679428,-0.03323399442551897,-0.0692948355017653,-0.1123588295651338,0.09912134350049173,-0.017358237998634892,0.07534173505750749,0.12767479534539006,-0.029835398734018057,-0.16110639840242838,-0.07185524539477744,0.0489826179897966,-0.03810787611728273,-0.023741612359666233,-0.11607006469274614,-0.021758453053172623,-0.06882503417754875,-0.06411639952880567,0.11201044693204698,0.03976999258796289,0.011005063045361016,-0.0628670226168293,-0.11922592681335077,0.017112437691080565,0.03274256848403089,-0.016637775719880035,-0.025338718067676916,-0.0005323320193269777,-0.17653930776717824,-0.04156968813352344,0.005523152506702004,0.17733866009345625,-0.0025748091435719386,0.006913432497590824,0.15716033354515563,-0.048946220466680464,0.07368511890653144,-0.09082449932279948,0.026352079194246272,-0.13967052240310188,-0.12912395365071486,-0.06408021520318442,0.005130956941998728,-0.156692014317866,-0.08132681856742152,-0.0679060919077

In [14]:
prefix = 'sagemaker/xgboost/data'

# The Amazon SageMaker implementation of XGBoost supports CSV formats for training and inference
# For Training ContentType, valid inputs `text/csv`.

train_data_uri = sess.upload_data(path="./data/train_transformed.csv", key_prefix=prefix + "/training")
validation_data_uri = sess.upload_data(path="./data/validation_transformed.csv", key_prefix=prefix + "/validation")

print(train_data_uri)
print(validation_data_uri)

s3_input_train_data = sagemaker.s3_input(s3_data=train_data_uri, content_type='text/csv')
s3_input_validation_data = sagemaker.s3_input(s3_data=validation_data_uri, content_type='text/csv')

s3://sagemaker-us-east-1-835319576252/sagemaker/xgboost/data/training/train_transformed.csv
s3://sagemaker-us-east-1-835319576252/sagemaker/xgboost/data/validation/validation_transformed.csv


In [13]:
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri 

# get the URI for new container
builtin_container_uri = get_image_uri(region_name=region,                                
                                  repo_name='xgboost', 
                                  repo_version='0.90-2')

model_output_path = 's3://{}/sagemaker/xgboost/training-runs'.format(bucket)

xgb_estimator = sagemaker.estimator.Estimator(image_name=builtin_container_uri, 
                                              role=role, 
                                              hyperparameters={'objective':'binary:logistic',
                                                               'num_round': 1,
                                                               'max_depth': 5},
                                              train_instance_count=1, 
                                              train_instance_type='ml.m4.xlarge', 
                                              output_path=model_output_path, 
                                              sagemaker_session=sess)

xgb_estimator.fit({'train': s3_input_train_data,
                   'validation': s3_input_validation_data
                  }
                  #, wait=False
                 )

2020-03-01 23:48:07 Starting - Starting the training job...
2020-03-01 23:48:08 Starting - Launching requested ML instances......
2020-03-01 23:49:15 Starting - Preparing the instances for training......
2020-03-01 23:50:27 Downloading - Downloading input data
2020-03-01 23:50:27 Training - Downloading the training image...
2020-03-01 23:51:06 Uploading - Uploading generated training model
2020-03-01 23:51:06 Failed - Training job failed
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[23:50:56] 1355x300 matrix with 406500 

UnexpectedStatusException: Error for Training job sagemaker-xgboost-2020-03-01-23-48-07-080: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.6/site-packages/sagemaker_xgboost_container/algorithm_mode/train.py", line 196, in train_job
    verbose_eval=False)
  File "/miniconda3/lib/python3.6/site-packages/xgboost/training.py", line 216, in train
    xgb_model=xgb_model, callbacks=callbacks)
  File "/miniconda3/lib/python3.6/site-packages/xgboost/training.py", line 32, in _train_internal
    bst = Booster(params, [dtrain] + [d[0] for d in evals])
  File "/miniconda3/lib/python3.6/site-packages/xgboost/core.py", line 941, in __init__
    self._validate_features(d)
  File "/miniconda3/lib/python3.6/site-packages/xgboost/core.py", line 1690, in _validate_features
    data.feature_names))
ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15'

In [ ]:
training_job_name = xgb_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

In [ ]:
# TODO:  This is broken
#from sagemaker.xgboost import XGBoost

#xgb_estimator = XGBoost.attach(training_job_name=training_job_name)

In [17]:
# download the model artifact from AWS S3
!aws s3 cp $model_output_path/$training_job_name/output/model.tar.gz ./model/


usage: aws s3 cp <LocalPath> <S3Uri> or <S3Uri> <LocalPath> or <S3Uri> <S3Uri>
Error: Invalid argument type


In [11]:
import tarfile
import pickle as pkl


#opens the downloaded model artifcat and loads it as 'model' variable
tar = tarfile.open('./model/model.tar.gz')
tar.extractall()
tar.close()
model = pkl.load(open('xgboost-model', 'rb'))


usage: aws s3 cp <LocalPath> <S3Uri> or <S3Uri> <LocalPath> or <S3Uri> <S3Uri>
Error: Invalid argument type


FileNotFoundError: [Errno 2] No such file or directory: 'model.tar.gz'

In [ ]:
import datetime as dt
import pickle as pkl
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


# From the following:  https://towardsdatascience.com/xgboost-in-amazon-sagemaker-28e5e354dbcd

map_names = dict(zip(model.feature_names, train_data[cols_input].columns))
model.feature_names = list(map_names.values())

#plot feature importance
fig, ax = plt.subplots(figsize=(12,12))
xgboost.plot_importance(model, importance_type='gain', max_num_features=30, height=0.8, ax=ax, show_values = False)
plt.title('Feature Importance')
plt.show()

# TODO: Deploy Endpoint

In [ ]:
# https://towardsdatascience.com/xgboost-in-amazon-sagemaker-28e5e354dbcd
from sagemaker.predictor import csv_serializer


## Deploy trained XGBoost model endpoint to perform predictions
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

## ?? Function to chunk down test set into smaller increments

# TODO:  1) update this to do TF/IDF
#        2) use this in other versions of the model

def predict(data, model, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, model.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

## Generate predictions on the test set for the difference models

predictions = predict(test_data[cols_input].values, xgb_predictor)

In [ ]:
# https://towardsdatascience.com/xgboost-in-amazon-sagemaker-28e5e354dbcd

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

def calc_specificity(y_true, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_true == 0)) /sum(y_true ==0)

thresh = 0.5
y_pred = predictions
y_pred_binary = np.where(predictions > thresh, 1, 0)
y_true = test_data['OUTPUT_LABEL']

c_mat = confusion_matrix(y_true, y_pred_binary) ## Predicted vs. actual outcome
auc = round(roc_auc_score(y_true, y_pred),4)
accuracy = round(accuracy_score(y_true,(y_pred > thresh) ) ,4)
recall = round(recall_score(y_true, (y_pred > thresh)),4)
precision = round(precision_score(y_true, (y_pred > thresh)),4)
specificity = round(calc_specificity(y_true, y_pred, thresh),4)

class_names = ['Not Readmitted', 'Readmitted'] ## Different class names

def plot_conf_mat(cm, classes, title, cmap = plt.cm.Blues):                                 
                                              
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
        horizontalalignment="center",
        color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

# Plot non-normalized confusion matrix
plt.figure()
fig, ax = plt.subplots(figsize=(6,4))
plot_conf_mat(c_mat, classes=class_names, 
                          title='Confusion matrix')
plt.show()
print(f'AUC is: {auc}')
print(f'Accuracy is: {accuracy}')
print(f'Recall is: {recall}')
print(f'Precision is: {precision}')
print(f'Specificity is: {specificity}')

In [ ]:
from sklearn import metrics

auc = round(roc_auc_score(y_true, y_pred), 4)
print('AUC is ' + repr(auc))

fpr, tpr, _ = metrics.roc_curve(y_true, y_pred)

plt.title('ROC Curve')
plt.plot(fpr, tpr, 'b',
label='AUC = %0.2f'% auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [10]:
# TODO:  1) update this to do TF/IDF
#        2) use this in other versions of the model
# Derived from the following:
#   https://aim357.readthedocs.io/en/latest/GluePySparkMLFeatureEngineering/GluePySparkMLFeatureEngineering.html#deepar-deep-dive

class XGBoostPredictor(sagemaker.predictor.RealTimePredictor):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, content_type=sagemaker.content_types.CONTENT_TYPE_JSON, **kwargs)

    def predict(self, ts, cat=None, dynamic_feat=None,
                num_samples=100, return_samples=False, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.

        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])

        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + 1
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)

    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None)

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles
        }

        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }

        return json.dumps(http_request_data).encode('utf-8')

    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode('utf-8'))['predictions'][0]
        prediction_length = len(next(iter(predictions['quantiles'].values())))
        prediction_index = pd.DatetimeIndex(start=prediction_time, freq=freq, periods=prediction_length)
        if return_samples:
            dict_of_samples = {'sample_' + str(i): s for i, s in enumerate(predictions['samples'])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(data={**predictions['quantiles'], **dict_of_samples}, index=prediction_index)

    def set_frequency(self, freq):
        self.freq = freq

def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]

def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat
    return obj

In [ ]:
xgb_endpoint_name = prefix + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

xgb_predictor = xgb_estimator.deploy(
                     initial_instance_count=1, 
                     instance_type='ml.m4.xlarge',
                     predictor_cls=XGBoostPredictor,
                     endpoint_name=xgb_endpoint_name)

In [ ]:
predictions, raw_outputs = model.predict(["""Very funny. A typical mid 50's comedy."""])
print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))

In [ ]:
predictions, raw_outputs = bert_model.predict(["""That movie was absolutely awful."""])
print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))